In [ ]:
import pandas as pd

df = pd.read_parquet('files_parquet/df_baseFinal.parquet', engine='pyarrow')
df[['temporada', 'nombre_categoria_producto', 'peso_producto_g', 
            'largo_producto_cm', 'altura_producto_cm', 'ancho_producto_cm', 
            'ciudad_cliente', 'estado_cliente',"id_producto"]]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Cargar la base de datos
df = pd.read_parquet('files_parquet/df_baseFinal.parquet', engine='pyarrow')

# Selección de características relevantes para la predicción
features = ['temporada', 'nombre_categoria_producto', 'peso_producto_g', 
            'largo_producto_cm', 'altura_producto_cm', 'ancho_producto_cm', 
            'ciudad_cliente', 'estado_cliente', "id_producto"]
target = 'precio'

# Preparar datos
X = df[features]
y = df[target]

# Preprocesamiento de datos
numerical_features = ['peso_producto_g', 'largo_producto_cm', 'altura_producto_cm', 'ancho_producto_cm']
categorical_features = ['temporada', 'nombre_categoria_producto', 'ciudad_cliente', 'estado_cliente', "id_producto"]

# Transformaciones
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Construcción del modelo con pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(random_state=42))
])

# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir parámetros para la búsqueda de hiperparámetros con GridSearchCV
param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__learning_rate': [0.01, 0.05, 0.1],
    'regressor__max_depth': [3, 6, 9],
    'regressor__subsample': [0.7, 0.8, 0.9],
    'regressor__colsample_bytree': [0.7, 0.8, 0.9],
    'regressor__gamma': [0, 0.1, 0.2],
    'regressor__reg_alpha': [0, 0.01, 0.1]
}

# Búsqueda de hiperparámetros con validación cruzada
grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Entrenamiento del modelo con GridSearchCV
grid_search.fit(X_train, y_train)

# Mejor modelo y sus parámetros
best_model = grid_search.best_estimator_
print(f"Best parameters found: {grid_search.best_params_}")

# Predicción con el mejor modelo
y_pred = best_model.predict(X_test)

# Calcular MSE (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)

# Calcular RMSE (Root Mean Squared Error)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

# Calcular R² (Coeficiente de Determinación)
r2 = r2_score(y_test, y_pred)
print(f"R²: {r2}")

# Evaluación con validación cruzada
cross_val_rmse = np.sqrt(-cross_val_score(best_model, X, y, cv=5, scoring='neg_mean_squared_error').mean())
print(f"Cross-validated RMSE: {cross_val_rmse}")



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Cargar la base de datos
df = pd.read_parquet('files_parquet/df_baseFinal.parquet', engine='pyarrow')

# Selección de características relevantes para la predicción
features = ['temporada', 'nombre_categoria_producto', 'peso_producto_g', 
            'largo_producto_cm', 'altura_producto_cm', 'ancho_producto_cm', 
            'ciudad_cliente', 'estado_cliente', "id_producto"]
target = 'precio'

# Preparar datos
X = df[features]
y = df[target]

# Preprocesamiento de datos
numerical_features = ['peso_producto_g', 'largo_producto_cm', 'altura_producto_cm', 'ancho_producto_cm']
categorical_features = ['temporada', 'nombre_categoria_producto', 'ciudad_cliente', 'estado_cliente', "id_producto"]

# Transformaciones
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Construcción del modelo con pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir parámetros para la búsqueda de hiperparámetros con RandomizedSearchCV
param_dist = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4],
    'regressor__max_features': ['sqrt', 'log2'],  
    'regressor__bootstrap': [True, False]
}

# Búsqueda de hiperparámetros con RandomizedSearchCV
random_search = RandomizedSearchCV(
    model, param_distributions=param_dist, n_iter=50, cv=5, n_jobs=-1, verbose=2, random_state=42,
    scoring='neg_mean_squared_error'
)

# Entrenamiento del modelo con RandomizedSearchCV
random_search.fit(X_train, y_train)

# Mejor modelo y sus parámetros
best_model = random_search.best_estimator_
print(f"Best parameters found: {random_search.best_params_}")

# Predicción con el mejor modelo
y_pred = best_model.predict(X_test)

# Calcular MSE (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)

# Calcular RMSE (Root Mean Squared Error)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Calcular MAE (Mean Absolute Error)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

# Calcular R² (Coeficiente de Determinación)
r2 = r2_score(y_test, y_pred)
print(f"R²: {r2}")

# Evaluación con validación cruzada
cross_val_rmse = np.sqrt(-cross_val_score(best_model, X, y, cv=5, scoring='neg_mean_squared_error').mean())
print(f"Cross-validated RMSE: {cross_val_rmse}")

# Ejemplo de predicción para un nuevo dato
new_data = pd.DataFrame({
    'temporada': ['Invierno'],
    'nombre_categoria_producto': ['cama_mesa_banho'],
    'id_producto': ['364e789259da982f5b7e43aaea7be61'],
    'peso_producto_g': [750],
    'largo_producto_cm': [16],
    'altura_producto_cm': [10],
    'ancho_producto_cm': [16],
    'ciudad_cliente': ['sao_paulo'],
    'estado_cliente': ['SP']
})
predicted_price = best_model.predict(new_data)
print(f"Predicted Seasonal Price: {predicted_price[0]}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Cargar la base de datos
df = pd.read_parquet('files_parquet/df_baseFinal.parquet', engine='pyarrow')

# Selección de características relevantes para la predicción
features = ['temporada', 'nombre_categoria_producto', 'peso_producto_g', 
            'largo_producto_cm', 'altura_producto_cm', 'ancho_producto_cm', 
            'ciudad_cliente', 'estado_cliente', "id_producto"]
target = 'precio'

# Preparar datos
X = df[features]
y = df[target]

# Preprocesamiento de datos
numerical_features = ['peso_producto_g', 'largo_producto_cm', 'altura_producto_cm', 'ancho_producto_cm']
categorical_features = ['temporada', 'nombre_categoria_producto', 'ciudad_cliente', 'estado_cliente', "id_producto"]

# Transformaciones
preprocessor = ColumnTransformer(
    transformers=[ 
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Construcción del modelo con pipeline usando regresión lineal
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo
model.fit(X_train, y_train)

# Predicción con el modelo entrenado
y_pred = model.predict(X_test)

# Calcular MSE (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)

# Calcular RMSE (Root Mean Squared Error)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Calcular MAE (Mean Absolute Error)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

# Calcular R² (Coeficiente de Determinación)
r2 = r2_score(y_test, y_pred)
print(f"R²: {r2}")

# Evaluación con validación cruzada
cross_val_rmse = np.sqrt(-cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error').mean())
print(f"Cross-validated RMSE: {cross_val_rmse}")

# Ejemplo de predicción para un nuevo dato
new_data = pd.DataFrame({
    'temporada': ['Invierno'],
    'nombre_categoria_producto': ['cama_mesa_banho'],
    'id_producto': ['364e789259da982f5b7e43aaea7be61'],
    'peso_producto_g': [750],
    'largo_producto_cm': [16],
    'altura_producto_cm': [10],
    'ancho_producto_cm': [16],
    'ciudad_cliente': ['sao_paulo'],
    'estado_cliente': ['SP']
})
predicted_price = model.predict(new_data)
print(f"Predicted Seasonal Price: {predicted_price[0]}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Cargar la base de datos
df = pd.read_parquet('files_parquet/df_baseFinal.parquet', engine='pyarrow')

# Selección de características relevantes para la predicción
features = ['temporada', 'nombre_categoria_producto', 'peso_producto_g', 
            'largo_producto_cm', 'altura_producto_cm', 'ancho_producto_cm', 
            'ciudad_cliente', 'estado_cliente', "id_producto"]
target = 'precio'

# Preparar datos
X = df[features]
y = df[target]

# Preprocesamiento de datos
numerical_features = ['peso_producto_g', 'largo_producto_cm', 'altura_producto_cm', 'ancho_producto_cm']
categorical_features = ['temporada', 'nombre_categoria_producto', 'ciudad_cliente', 'estado_cliente', "id_producto"]

# Transformaciones
preprocessor = ColumnTransformer(
    transformers=[ 
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Construcción del modelo con pipeline usando ElasticNet
model = Pipeline(steps=[ 
    ('preprocessor', preprocessor),
    ('regressor', ElasticNet())
])

# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo
model.fit(X_train, y_train)

# Predicción con el modelo entrenado
y_pred = model.predict(X_test)

# Calcular MSE (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)

# Calcular RMSE (Root Mean Squared Error)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Calcular MAE (Mean Absolute Error)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

# Calcular R² (Coeficiente de Determinación)
r2 = r2_score(y_test, y_pred)
print(f"R²: {r2}")

# Evaluación con validación cruzada
cross_val_rmse = np.sqrt(-cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error').mean())
print(f"Cross-validated RMSE: {cross_val_rmse}")

# Ejemplo de predicción para un nuevo dato
new_data = pd.DataFrame({
    'temporada': ['Invierno'],
    'nombre_categoria_producto': ['cama_mesa_banho'],
    'id_producto': ['364e789259da982f5b7e43aaea7be61'],
    'peso_producto_g': [750],
    'largo_producto_cm': [16],
    'altura_producto_cm': [10],
    'ancho_producto_cm': [16],
    'ciudad_cliente': ['sao_paulo'],
    'estado_cliente': ['SP']
})
predicted_price = model.predict(new_data)
print(f"Predicted Seasonal Price: {predicted_price[0]}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Cargar la base de datos
df = pd.read_parquet('files_parquet/df_baseFinal.parquet', engine='pyarrow')

# Selección de características relevantes para la predicción
features = ['temporada', 'nombre_categoria_producto', 'peso_producto_g', 
            'largo_producto_cm', 'altura_producto_cm', 'ancho_producto_cm', 
            'ciudad_cliente', 'estado_cliente', "id_producto"]
target = 'precio'

# Preparar datos
X = df[features]
y = df[target]

# Preprocesamiento de datos
numerical_features = ['peso_producto_g', 'largo_producto_cm', 'altura_producto_cm', 'ancho_producto_cm']
categorical_features = ['temporada', 'nombre_categoria_producto', 'ciudad_cliente', 'estado_cliente', "id_producto"]

# Transformaciones
preprocessor = ColumnTransformer(
    transformers=[ 
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Construcción del modelo con pipeline usando K-Nearest Neighbors
model = Pipeline(steps=[ 
    ('preprocessor', preprocessor),
    ('regressor', KNeighborsRegressor())
])

# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajuste de hiperparámetros con GridSearchCV
param_grid = {
    'regressor__n_neighbors': [3, 5, 7, 10],  # Número de vecinos
    'regressor__weights': ['uniform', 'distance'],  # Cómo ponderar los vecinos
    'regressor__p': [1, 2]  # Normas para la distancia (1=Manhattan, 2=Euclidiana)
}
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros
print(f"Best parameters found: {grid_search.best_params_}")

# Predicción con el modelo ajustado
y_pred = grid_search.best_estimator_.predict(X_test)

# Calcular MSE (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)

# Calcular RMSE (Root Mean Squared Error)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Calcular MAE (Mean Absolute Error)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

# Calcular R² (Coeficiente de Determinación)
r2 = r2_score(y_test, y_pred)
print(f"R²: {r2}")

# Evaluación con validación cruzada
cross_val_rmse = np.sqrt(-cross_val_score(grid_search.best_estimator_, X, y, cv=5, scoring='neg_mean_squared_error').mean())
print(f"Cross-validated RMSE: {cross_val_rmse}")

# Ejemplo de predicción para un nuevo dato
new_data = pd.DataFrame({
    'temporada': ['Invierno'],
    'nombre_categoria_producto': ['cama_mesa_banho'],
    'id_producto': ['364e789259da982f5b7e43aaea7be61'],
    'peso_producto_g': [750],
    'largo_producto_cm': [16],
    'altura_producto_cm': [10],
    'ancho_producto_cm': [16],
    'ciudad_cliente': ['sao_paulo'],
    'estado_cliente': ['SP']
})
predicted_price = grid_search.best_estimator_.predict(new_data)
print(f"Predicted Seasonal Price: {predicted_price[0]}")


In [ ]:
#Comparando modelos
import matplotlib.pyplot as plt
import numpy as np

# Nombres de los modelos
model_names = [
    "XGBRegressor", 
    "RandomForestRegressor", 
    "LinearRegression", 
    "ElasticNet", 
    "KNeighborsRegressor"
]

# Métricas de cada modelo
rmse_scores = [
    132.67,  # XGBRegressor
    135.53,  # RandomForestRegressor
    118.01,  # LinearRegression
    183.53,    # ElasticNet
    142.54   # KNeighborsRegressor
]

mae_scores = [
    54.76,   # XGBRegressor
    43.09,   # RandomForestRegressor
    25.55,   # LinearRegression
    81.10,     # ElasticNet
    39.61    # KNeighborsRegressor
]

r2_scores = [
    0.54,    # XGBRegressor
    0.52,    # RandomForestRegressor
    0.63,    # LinearRegression
    0.11,      # ElasticNet
    0.47     # KNeighborsRegressor
]

# Ordenar las métricas y modelos de mayor a menor
rmse_order = np.argsort(rmse_scores)[::-1]
mae_order = np.argsort(mae_scores)[::-1]
r2_order = np.argsort(r2_scores)[::-1]

# Aplicar el orden a cada lista
rmse_scores_sorted = [rmse_scores[i] for i in rmse_order]
mae_scores_sorted = [mae_scores[i] for i in mae_order]
r2_scores_sorted = [r2_scores[i] for i in r2_order]

model_names_rmse_sorted = [model_names[i] for i in rmse_order]
model_names_mae_sorted = [model_names[i] for i in mae_order]
model_names_r2_sorted = [model_names[i] for i in r2_order]

# Crear subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Comparación de Modelos (Ordenados de Mayor a Menor)')

# Gráfico RMSE
axes[0].bar(model_names_rmse_sorted, rmse_scores_sorted, color='skyblue')
axes[0].set_title('RMSE')
axes[0].set_ylabel('Root Mean Squared Error')
axes[0].set_xticklabels(model_names_rmse_sorted, rotation=45)

# Gráfico MAE
axes[1].bar(model_names_mae_sorted, mae_scores_sorted, color='lightgreen')
axes[1].set_title('MAE')
axes[1].set_ylabel('Mean Absolute Error')
axes[1].set_xticklabels(model_names_mae_sorted, rotation=45)

# Gráfico R²
axes[2].bar(model_names_r2_sorted, r2_scores_sorted, color='salmon')
axes[2].set_title('R²')
axes[2].set_ylabel('Coeficiente de Determinación')
axes[2].set_xticklabels(model_names_r2_sorted, rotation=45)

# Mostrar los gráficos
plt.tight_layout()
plt.subplots_adjust(top=0.88)
plt.show()


LinearRegression es claramente el mejor modelo según estas métricas. Tiene los valores de RMSE y MAE más bajos y el valor de R² más alto, lo cual indica que es el modelo más preciso y confiable para este conjunto de datos. Por otro lado, ElasticNet es el menos efectivo, mostrando el peor rendimiento en todas las métricas. Si deseas una alternativa a LinearRegression, XGBRegressor podría ser una opción viable, aunque su rendimiento es inferior.

In [7]:
#Prediccion y sacando la base prueba para el dash
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Cargar la base de datos
df = pd.read_parquet('files_parquet/df_baseFinal.parquet', engine='pyarrow')

# Selección de características relevantes para la predicción
features = ['temporada', 'nombre_categoria_producto', 'peso_producto_g', 
            'largo_producto_cm', 'altura_producto_cm', 'ancho_producto_cm', 
            'ciudad_cliente', 'estado_cliente', "id_producto"]
target = 'precio'

# Preparar datos
X = df[features]
y = df[target]

# Preprocesamiento de datos
numerical_features = ['peso_producto_g', 'largo_producto_cm', 'altura_producto_cm', 'ancho_producto_cm']
categorical_features = ['temporada', 'nombre_categoria_producto', 'ciudad_cliente', 'estado_cliente', "id_producto"]

# Transformaciones
preprocessor = ColumnTransformer(
    transformers=[ 
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Construcción del modelo con pipeline usando regresión lineal
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo
model.fit(X_train, y_train)

# Predicción con el modelo entrenado en el conjunto completo de datos
df['predicted_price'] = model.predict(X)

# Guardar el DataFrame con la columna de predicción añadida
df.to_csv('files_csv/base_dash.csv')
print("Archivo base_dash.csv guardado exitosamente con las predicciones incluidas.")


Archivo base_dash.csv guardado exitosamente con las predicciones incluidas.


###